¿Que hace este script?
* Calcula para cada dispositivo de red: trafico.totales.traficoIN, trafico.totales.traficoOUT', trafico.totales.consumoGB'.
* Calcula para cada site_id: trafico.totales.sitio.traficoIN, trafico.totales.sitio.traficoOUT, trafico.totales.sitio.consumoGB

In [1]:
from elasticsearch import Elasticsearch, helpers
from ssl import create_default_context
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import parametros
import random
import re

## Conectando a ElasticSearch

La ultima línea se utiliza para garantizar la ejecución de la consulta
* timeout es el tiempo para cada ejecución
* max_retries el número de intentos si la conexión falla
* retry_on_timeout para activar los reitentos

In [2]:
context = create_default_context(cafile=parametros.cafile)
es = Elasticsearch(
    parametros.servidor,
    http_auth=(parametros.usuario_EC, parametros.password_EC),
    scheme="https",
    port=parametros.puerto,
    ssl_context=context,
    timeout=60, max_retries=3, retry_on_timeout=True
) 

### Calculando fechas para la ejecución

* Se calculan las fechas para asociar al nombre del indice
* fecha_hoy es usada para concatenar al nombre del indice principal previa inserción

In [3]:
now = datetime.now()
fecha_hoy = str(now.strftime("%Y.%m.%d"))

### Definiendo indice principal con fecha de hoy

Estos valores se deben ajustar según ambiente. No es automático ya que no hay separación de ambientes

In [4]:
indice = parametros.trafico_tablero05_index
indice_control = parametros.tableros_mintic_control

### Función para generar JSON compatible con ES

In [5]:
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

### leyendo indice semilla-inventario

En el script que ingesta semilla, trae la información de los centros de conexión administrados. Para el indice principal se requiere:

* site_id como llave del centro de conexión.
* Datos geográficos (Departamento, municipio, centro poblado, sede, energía, latitud, longitud, COD_ISO, entre otros).

In [6]:
total_docs = 10000
try:
    response = es.search(
        index= parametros.semilla_inventario_index,
        body={
               "_source": ['site_id','nombre_municipio', 'nombre_departamento', 'nombre_centro_pob', 'nombreSede' 
                           , 'energiadesc', 'latitud', 'longitud','COD_ISO','id_Beneficiario']
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    datos_semilla = pd.DataFrame([x["_source"] for x in elastic_docs])
    
except:
    print("fecha:",now,"- Error en lectura de datos semilla")
    #exit()
    

In [7]:
datos_semilla['site_id'] = datos_semilla['site_id'].str.strip()

Se valida latitud y longitud, se genera campo location y se renombran los campos de semilla

In [8]:
def get_location(x,y='lat'):
    patron = re.compile('^(\-?\d+(\.\d+)?),\s*(\-?\d+(\.\d+)?)$') #patrón que debe cumplir
    if (not patron.match(x) is None) and (str(x)!=''):
        return x.replace(',','.')
    else:
        #Código a ejecutar si las coordenadas no son válidas
        return '4.596389' if y=='lat' else '-74.074639'
    
datos_semilla['latitud'] = datos_semilla['latitud'].apply(lambda x:get_location(x,'lat'))
datos_semilla['longitud'] = datos_semilla['longitud'].apply(lambda x:get_location(x,'lon'))
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["longitud"]=='a') | (datos_semilla["latitud"]=='a')].index)
datos_semilla['trafico.tablero05.location'] = datos_semilla['latitud'] + ',' + datos_semilla['longitud']
datos_semilla['trafico.tablero05.location']=datos_semilla['trafico.tablero05.location'].str.replace('a,a','')
datos_semilla.drop(columns=['latitud','longitud'],inplace=True)

datos_semilla = datos_semilla.rename(columns={'nombre_municipio': 'trafico.tablero05.nombreMunicipio'
                                              , 'nombre_departamento' : 'trafico.tablero05.nombreDepartamento'
                                              , 'nombre_centro_pob': 'trafico.tablero05.localidad'
                                              , 'nombreSede' : 'trafico.tablero05.nomCentroDigital'
                                              , 'energiadesc' : 'trafico.tablero05.sistemaEnergia'
                                              , 'COD_ISO' : 'trafico.tablero05.codISO'
                                              , 'id_Beneficiario' : 'trafico.tablero05.idBeneficiario'})
datos_semilla.fillna('', inplace=True)

In [9]:
datos_semilla = datos_semilla.drop(datos_semilla[(datos_semilla["trafico.tablero05.location"]=='')].index)

### leyendo indice cambium-devicedevices

Se lee la información de los dispositivos de red monitoreados por Cambium. En esta lectura no hay referencia de fechas ya que solo hay una ocurrencia por MAC de dispositivo de red.

* site_id es la llave para cruzar con cada centro de conexión.
* mac, IP y name son datos básicos del dispositivo.
* ap_group identifica los dispositivos como INDOOR u OUTDOOR

In [10]:
total_docs = 30000
try:
    response = es.search(
        index= parametros.cambium_d_d_index,
        body={
                    "_source": ["site_id","mac",'name',"ip","ap_group"]
                  , "query": {
                    "match_all": {}
                  }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    datos_dev = pd.DataFrame([x["_source"] for x in elastic_docs])
    
    
except:
    exit()

In [11]:
datos_dev['site_id'] = datos_dev['site_id'].str.strip()

Se descartan registros con site_id vacios y se limpian los NaN del dataframe

In [12]:
datos_dev.dropna(subset=['site_id'])
datos_dev.fillna('', inplace=True)
datos_dev = datos_dev.drop(datos_dev[(datos_dev['site_id']=='')].index)
datos_dev.sort_values(['site_id','ap_group'], inplace=True)

Se limpian datos mal formados de ap_group

In [13]:
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("-", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split("_", n = 1, expand = True)[0]
datos_dev['ap_group'] = datos_dev['ap_group'].str.split(".", n = 1, expand = True)[0]
datos_dev = datos_dev.drop(datos_dev[(datos_dev['ap_group']=='')].index)

In [14]:
datos_dev = datos_dev.drop_duplicates('mac')

Se renombran campos según formato del indice final

In [15]:
datos_dev = datos_dev.rename(columns={'ap_mac' : 'trafico.tablero05.macRed'
                                        ,'ap_group': 'trafico.tablero05.apGroup'
                                        , 'ip': 'trafico.tablero05.IP'
                                        , 'mac' : 'trafico.tablero05.macRed'
                                        , 'name' : 'trafico.tablero05.deviceName'})

### Trae la ultima fecha para control de ejecución

Cuando en el rango de tiempo de la ejecución, no se insertan nuevos valores, las fecha maxima en indice mintic no aumenta, por tanto se usa esta fecha de control para garantizar que incremente el bucle de ejecución

In [16]:
total_docs = 1
try:
    response = es.search(
        index= indice_control,
        body={
               "_source": ["tablero05.fechaControl"],
              "query": {
                "bool": {
                  "filter": [
                  {
                    "exists": {
                      "field":"jerarquia-tablero05"
                    }
                  }
                  ]
                }
              }
        },
        size=total_docs
    )
    #print(es.info())
    elastic_docs = response["hits"]["hits"]
    fields = {}
    for num, doc in enumerate(elastic_docs):
        fecha_ejecucion = doc["_source"]['tablero05.fechaControl']
except:
    fecha_ejecucion = '2021-05-01 00:00:00'
if response["hits"]["hits"] == []:
    fecha_ejecucion = '2021-05-01 00:00:00'
print("ultima fecha para control de ejecucion:",fecha_ejecucion)

ultima fecha para control de ejecucion: 2021-06-01 00:00:00


## Se lee la información de cambium device performance

 Se toma los valores de dispositivos de red y su desempeño.
 * mac del dispositivo de red
 * timestamp es la fecha y hora de la medición
 * radio.* volumen de datos descargados(r) y cargados(t)

In [17]:
def traePerformance(fecha_max,fecha_tope):
    total_docs = 5000000
    response = es.search(
        index= parametros.cambium_d_p_index,
        body={
                "_source": ["mac","timestamp","radio.5ghz.rx_bps",
                           "radio.5ghz.tx_bps","radio.24ghz.rx_bps"
                          ,"radio.24ghz.tx_bps"]
              , "query": {
                  "range": {
                    "timestamp": {
                      "gte": fecha_max,
                      "lt": fecha_tope
                    }
                  }
              }
        },
        size=total_docs
    )
    elastic_docs = response["hits"]["hits"]

    return pd.DataFrame([x["_source"] for x in elastic_docs])

### Lanzando  ejecución de consulta

* Se calcula rango en base a la fecha de control. Para este caso es de 10 minutos.
* Se ejecuta la función de consulta con el rango de fechas.
* Si no retorna datos se incrementa el rango y se ejecuta nuevamente. Este proceso se repite hasta conseguir datos o hasta que el rango de ejecución alcance la fecha y hora actual.

In [18]:
fecha_max_mintic = fecha_ejecucion
fecha_tope_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)-timedelta(seconds=1)).strftime("%Y-%m-%d %H:%M:%S")
datos_performance = traePerformance(fecha_max_mintic,fecha_tope_mintic)

if datos_performance is None or datos_performance.empty:
    while (datos_performance is None or datos_performance.empty) and ((datetime.strptime(fecha_max_mintic[0:10], '%Y-%m-%d').strftime("%Y-%m-%d %H:%M:%S")) < str(now.strftime("%Y-%m-%d %H:%M:%S"))):
        fecha_max_mintic = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        fecha_tope_mintic = (datetime.strptime(fecha_tope_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")
        datos_performance = traePerformance(fecha_max_mintic,fecha_tope_mintic)
else:
    pass

In [19]:
#datos_performance = traePerformance('2021-06-10 09:00:00','2021-06-10 10:00:00')

funcion para insertar en indice: 

In [20]:
use_these_keys = ['trafico.tablero05.nomCentroDigital',
                  'trafico.tablero05.codISO',
                  'trafico.tablero05.localidad',
                  'trafico.tablero05.siteID',
                  'trafico.tablero05.nombreDepartamento',
                  'trafico.tablero05.sistemaEnergia',
                  'trafico.tablero05.nombreMunicipio',
                  'trafico.tablero05.idBeneficiario',
                  'trafico.tablero05.apGroup',
                  'trafico.tablero05.macRed',
                  'trafico.tablero05.IP',
                  'trafico.tablero05.deviceName',
                  'trafico.tablero05.macRed',
                  'trafico.totales.tablero05.fechaControl',
                  'trafico.totales.tablero05.traficoIN',
                  'trafico.totales.tablero05.traficoOUT',
                  'trafico.totales.tablero05.consumoGB',
                  'trafico.totales.tablero05.fecha',
                  'trafico.totales.tablero05.anyo',
                  'trafico.totales.tablero05.mes',
                  'trafico.totales.tablero05.dia',
                  'trafico.totales.tablero05.hora',
                  'trafico.totales.tablero05.minuto',
                  'trafico.totales.promedioIN',
                  'trafico.totales.promedioOUT',
                  'trafico.totales.promedioConsumo',
                  'nombreDepartamento',
                    'nombreMunicipio',
                    'idBeneficiario',
                    'fecha',
                    'anyo',
                    'mes',
                    'dia',
                  '@timestamp']
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'consumo-' + document['trafico.tablero05.siteID'] + '-' + document['trafico.tablero05.macRed'] + '-' +document['trafico.totales.tablero05.fechaControl']}",
                "_source": filterKeys(document),
            }


# Insertando consumo a indice 

* Calculo de indicadores de consumos para cada AP. 
* Se toman los valores de rx y tx como descarga y carga
* de datos_dev se toma el site_id y el ap_group

In [21]:
#try:
datos_performance['fecha_control'] = datos_performance["timestamp"].str[0:-4] + '0:00'
datos_performance.rename(columns={'mac': 'trafico.tablero05.macRed'}, inplace=True)
datos_performance.replace('','0',inplace=True)
datos_performance.fillna({'radio.5ghz.rx_bps':0, 'radio.5ghz.tx_bps':0,
                    'radio.24ghz.rx_bps':0, 'radio.24ghz.tx_bps':0 },inplace=True)
datos_performance[['radio.5ghz.rx_bps','radio.5ghz.tx_bps','radio.24ghz.rx_bps','radio.24ghz.tx_bps']] = datos_performance[['radio.5ghz.rx_bps','radio.5ghz.tx_bps','radio.24ghz.rx_bps','radio.24ghz.tx_bps']].astype(int)
    
       
aux_performance=datos_performance[['trafico.tablero05.macRed','fecha_control'
                                     ,'radio.5ghz.rx_bps'
                                     ,'radio.5ghz.tx_bps'
                                    ,'radio.24ghz.rx_bps'
                                     ,'radio.24ghz.tx_bps']].groupby(['trafico.tablero05.macRed','fecha_control']).agg(['sum']).reset_index()
aux_performance.columns = aux_performance.columns.droplevel(1)
aux_performance['trafico.totales.tablero05.traficoIN_aux'] = aux_performance['radio.5ghz.rx_bps'] + aux_performance['radio.24ghz.rx_bps']
aux_performance['trafico.totales.tablero05.traficoOUT_aux'] = aux_performance['radio.5ghz.tx_bps'] + aux_performance['radio.24ghz.tx_bps']
    
aux_performance = pd.merge(aux_performance, datos_dev, on='trafico.tablero05.macRed',how='inner')
mintic_02 = pd.merge(datos_semilla,  aux_performance, on='site_id',how='inner')
    
    ## Calculo del promedio 
aux_performance['trafico.totales.promedioIN'] = round((aux_performance['trafico.totales.tablero05.traficoIN_aux']/float(1<<30)),6)
aux_performance['trafico.totales.promedioOUT'] = round((aux_performance['trafico.totales.tablero05.traficoOUT_aux']/float(1<<30)),6)
aux_performance['trafico.totales.promedioConsumo'] = aux_performance['trafico.totales.promedioIN'] + aux_performance['trafico.totales.promedioOUT']
    #Se totaliza para la sede completa
aux_performance['trafico.totales.promedioConsumo'] = round(aux_performance['trafico.totales.promedioConsumo'],6)

       
    #La información de tráfico se convierte a GigaBytes y se toman 6 decimales
mintic_02['trafico.totales.tablero05.traficoIN'] = round((mintic_02['trafico.totales.tablero05.traficoIN_aux']/float(1<<30)),6)
mintic_02['trafico.totales.tablero05.traficoOUT'] = round((mintic_02['trafico.totales.tablero05.traficoOUT_aux']/float(1<<30)),6)
mintic_02['trafico.totales.tablero05.consumoGB'] = mintic_02['trafico.totales.tablero05.traficoIN'] + mintic_02['trafico.totales.tablero05.traficoOUT']
    #Se totaliza entrante y saliente
mintic_02['trafico.totales.tablero05.consumoGB'] = round(mintic_02['trafico.totales.tablero05.consumoGB'],6)

    ### Generando columnas con fecha, anyo, mes, dia, hora y minuto por separado
mintic_02["trafico.totales.tablero05.fecha"] = mintic_02["fecha_control"].str.split(" ", n = 1, expand = True)[0]
mintic_02["trafico.totales.tablero05.hora"] = mintic_02["fecha_control"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[0]
mintic_02["trafico.totales.tablero05.minuto"] = mintic_02["fecha_control"].str.split(" ", n = 1, expand = True)[1].str.split(":", n = 2, expand = True)[1]
mintic_02["trafico.totales.tablero05.anyo"] = mintic_02["trafico.totales.tablero05.fecha"].str[0:4]
mintic_02["trafico.totales.tablero05.mes"] = mintic_02["trafico.totales.tablero05.fecha"].str[5:7]
mintic_02["trafico.totales.tablero05.dia"] = mintic_02["trafico.totales.tablero05.fecha"].str[8:10]
    ### Renombrado de campos
mintic_02.rename(columns={'site_id': 'trafico.tablero05.siteID'
                            ,'fecha_control' : 'trafico.totales.tablero05.fechaControl'}, inplace=True)
    
aux_performance=aux_performance.rename(columns={'site_id' : 'trafico.tablero05.siteID', \
                                                'fecha_control':'trafico.totales.tablero05.fechaControl'})

    ##nulos a cero
mintic_02.fillna({'trafico.totales.tablero05.consumoGB':0,
                    'trafico.totales.tablero05.traficoIN':0,
                    'trafico.totales.tablero05.traficoOUT':0
                    },inplace=True)
    #cambia valores a tipo float
mintic_02[['trafico.totales.tablero05.consumoGB','trafico.totales.tablero05.traficoIN','trafico.totales.tablero05.traficoOUT']] = mintic_02[['trafico.totales.tablero05.consumoGB','trafico.totales.tablero05.traficoIN','trafico.totales.tablero05.traficoOUT']].astype(float)
    
mintic_02['nombreDepartamento'] = mintic_02['trafico.tablero05.nombreDepartamento']
mintic_02['nombreMunicipio'] = mintic_02['trafico.tablero05.nombreMunicipio']
mintic_02['idBeneficiario'] = mintic_02['trafico.tablero05.idBeneficiario']
mintic_02['fecha'] = mintic_02['trafico.totales.tablero05.fecha']
mintic_02['anyo'] = mintic_02['trafico.totales.tablero05.anyo']
mintic_02['mes'] = mintic_02['trafico.totales.tablero05.mes']
mintic_02['dia'] = mintic_02['trafico.totales.tablero05.dia']
mintic_02['@timestamp'] = now.isoformat()
    
aux_performance = aux_performance[['trafico.tablero05.siteID'
                              ,'trafico.tablero05.macRed'
                              ,'trafico.totales.tablero05.fechaControl'
                              ,'trafico.totales.promedioIN'
                              ,'trafico.totales.promedioOUT'
                              ,'trafico.totales.promedioConsumo']]
    
mintic_02 = pd.merge(mintic_02,aux_performance, 
                        on=['trafico.tablero05.siteID',
                            'trafico.tablero05.macRed',
                            'trafico.totales.tablero05.fechaControl'],how='left')

mintic_02.rename(columns={'trafico.tablero05.apGroup_x': 'trafico.tablero05.apGroup'}, inplace=True)
    
    #salida = helpers.bulk(es, doc_generator(mintic_02))
        
    #print("Fecha: ", now,"- Datos Trafico Consumos en indice principal:",salida[0])
#except:
    #print("Fecha: ", now,"- No se insertaron datos de consumos en indice principal")


## En otra  jerarquía se escriben los dispositivos conectados

Se toma el dataframe del proceso anterior para realizar el siguiente proceso:
* Se cuentan la cantidad de dispositivos WAN/LAN (OUTDOOR/INDOOR), agrupando por fecha y sitio
* Cantidad dispositivos WAN/LAN conectados se calcula validando cuando son OUTDOOR/INDOOR y tienen trafico(traficoOUT)
* Cantidad de dispositivos desconectados, restando los dos anteriores

Datos generados
* trafico.totales.cantDevWAN
* trafico.totales.cantDevLAN
* trafico.totales.cantDevConectadosWAN
* trafico.totales.cantDevDesconectadosWAN
* trafico.totales.cantDevConectadosLAN
* trafico.totales.cantDevDesconectadosLAN
* trafico.totales.cantDev

In [ ]:
try:
    mintic_03 = mintic_02[['trafico.tablero05.nomCentroDigital',
                      'trafico.tablero05.codISO',
                      'trafico.tablero05.localidad',
                      'trafico.tablero05.siteID',
                      'trafico.tablero05.nombreDepartamento',
                      'trafico.tablero05.sistemaEnergia',
                      'trafico.tablero05.nombreMunicipio',
                      'trafico.tablero05.idBeneficiario',
                      'trafico.tablero05.location',
                      'trafico.tablero05.apGroup',
                      'trafico.tablero05.macRed',
                      'trafico.totales.tablero05.fechaControl',
                      'trafico.totales.tablero05.fecha',
                      'trafico.totales.tablero05.anyo',
                      'trafico.totales.tablero05.mes',
                      'trafico.totales.tablero05.dia',
                      'trafico.totales.tablero05.hora',
                      'trafico.totales.tablero05.minuto',
                      'trafico.totales.tablero05.traficoIN',
                      'trafico.totales.tablero05.traficoOUT',
                      'trafico.totales.tablero05.consumoGB',
                      'trafico.totales.promedioIN',
                      'trafico.totales.promedioOUT',
                      'trafico.totales.promedioConsumo']].groupby(['trafico.tablero05.nomCentroDigital',
                                                              'trafico.tablero05.codISO',
                                                              'trafico.tablero05.localidad',
                                                              'trafico.tablero05.siteID',
                                                              'trafico.tablero05.nombreDepartamento',
                                                              'trafico.tablero05.sistemaEnergia',
                                                              'trafico.tablero05.nombreMunicipio',
                                                              'trafico.tablero05.idBeneficiario',
                                                              'trafico.tablero05.location',
                                                              'trafico.tablero05.apGroup',
                                                              'trafico.tablero05.macRed',
                                                              'trafico.totales.tablero05.fechaControl',
                                                              'trafico.totales.tablero05.fecha',
                                                              'trafico.totales.tablero05.anyo',
                                                              'trafico.totales.tablero05.mes',
                                                              'trafico.totales.tablero05.dia',
                                                              'trafico.totales.tablero05.hora',
                                                              'trafico.totales.promedioIN',
                                                              'trafico.totales.promedioOUT',
                                                              'trafico.totales.promedioConsumo',
                                                              'trafico.totales.tablero05.minuto']).agg(['sum']).reset_index()
    mintic_03.columns = mintic_03.columns.droplevel(1)
    mintic_03.rename(columns={'trafico.totales.tablero05.traficoIN' : 'trafico.totales.sitio.tablero05.traficoIN'
                          ,'trafico.totales.tablero05.traficoOUT' : 'trafico.totales.sitio.tablero05.traficoOUT'
                          ,'trafico.totales.tablero05.consumoGB' : 'trafico.totales.sitio.tablero05.consumoGB'
                         }, inplace=True)
except:
    pass

Funcion para insertar en indice la cantidad de dispositivos conectados 
* la lista use_these_keys se usa para referenciar cuales campos del dataframe irán al indice final. si los datos no se declaran en este, no se insertarán


In [ ]:
use_these_keys = ['trafico.nomCentroDigital',
                  'trafico.codISO',
                  'trafico.localidad',
                  'trafico.siteID',
                  'trafico.nombreDepartamento',
                  'trafico.sistemaEnergia',
                  'trafico.nombreMunicipio',
                  'trafico.idBeneficiario',
                  'trafico.location',
                  'trafico.apGroup',
                  'trafico.totales.fechaControl',
                  'trafico.totales.cantDevWAN',
                  'trafico.totales.cantDevConectadosWAN',
                  'trafico.totales.cantDevDesconectadosWAN',
                  'trafico.totales.cantDevLAN',
                  'trafico.totales.cantDevConectadosLAN',
                  'trafico.totales.cantDevDesconectadosLAN',
                  'trafico.totales.cantDev',
                  'trafico.totales.cantDevConectados',
                  'trafico.totales.cantDevDesconectados',
                  'trafico.totales.fecha',
                  'trafico.totales.anyo',
                  'trafico.totales.mes',
                  'trafico.totales.dia',
                  'trafico.totales.hora',
                  'trafico.totales.minuto',
                  'trafico.totales.sitio.traficoIN',
                  'trafico.totales.sitio.traficoOUT',
                  'trafico.totales.sitio.consumoGB',
                  'trafico.totales.promedioIN',
                  'trafico.totales.promedioOUT',
                  'trafico.totales.promedioConsumo',
                  '@timestamp']

def doc_generator_dis(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": indice, 
                "_id": f"{'Conectados-' + document['trafico.siteID'] + '-' +document['trafico.totales.fechaControl']+'-'+str(random.randrange(10000))}",
                "_source": filterKeys(document),
            }

# Insertando cantidad de dispositivos conectados

In [ ]:
try:
    cantDevWAN = mintic_02[(mintic_02['trafico.apGroup']=='OUTDOOR')][['trafico.totales.fechaControl','trafico.macRed','trafico.siteID']].groupby(['trafico.siteID','trafico.totales.fechaControl'])['trafico.macRed'].nunique().reset_index()    
    cantDevWAN.rename(columns={'trafico.macRed': 'trafico.totales.cantDevWAN'}, inplace=True)
    mintic_03 = pd.merge(mintic_03,  cantDevWAN, on=['trafico.siteID','trafico.totales.fechaControl'],how='left')

    cantDevConectadosWAN = mintic_02[(mintic_02['trafico.totales.traficoOUT']>0) & (mintic_02['trafico.apGroup']=='OUTDOOR')][['trafico.totales.fechaControl','trafico.macRed','trafico.siteID']].groupby(['trafico.siteID','trafico.totales.fechaControl'])['trafico.macRed'].nunique().reset_index()
    cantDevConectadosWAN.rename(columns={'trafico.macRed': 'trafico.totales.cantDevConectadosWAN'}, inplace=True)
    mintic_03 = pd.merge(mintic_03, cantDevConectadosWAN, on=['trafico.siteID','trafico.totales.fechaControl'],how='left')
    mintic_03.fillna({'trafico.totales.cantDevWAN':0,
                      'trafico.totales.cantDevConectadosWAN':0
                      },inplace=True)
    mintic_03['trafico.totales.cantDevDesconectadosWAN'] = mintic_03['trafico.totales.cantDevWAN'] - mintic_03['trafico.totales.cantDevConectadosWAN']

    #La misma lógica se aplica para calcular las cantidades para dispositivos LAN, pero filtrando los INDOOR
    cantDevLAN = mintic_02[(mintic_02['trafico.apGroup']=='INDOOR')][['trafico.totales.fechaControl','trafico.macRed','trafico.siteID']].groupby(['trafico.siteID','trafico.totales.fechaControl'])['trafico.macRed'].nunique().reset_index()
    cantDevLAN.rename(columns={'trafico.macRed': 'trafico.totales.cantDevLAN'}, inplace=True)
    mintic_03 = pd.merge(mintic_03,  cantDevLAN, on=['trafico.siteID','trafico.totales.fechaControl'],how='left')

    cantDevConectadosLAN = mintic_02[~(mintic_02['trafico.totales.traficoIN']>0) & (mintic_02['trafico.apGroup']=='INDOOR')][['trafico.totales.fechaControl','trafico.macRed','trafico.siteID']].groupby(['trafico.siteID','trafico.totales.fechaControl'])['trafico.macRed'].nunique().reset_index()
    cantDevConectadosLAN.rename(columns={'trafico.macRed': 'trafico.totales.cantDevConectadosLAN'}, inplace=True)

    mintic_03 = pd.merge(mintic_03, cantDevConectadosLAN, on=['trafico.siteID','trafico.totales.fechaControl'],how='left')
    mintic_03.fillna({'trafico.totales.cantDevLAN':0,
                      'trafico.totales.cantDevConectadosLAN':0
                      },inplace=True)
    mintic_03['trafico.totales.cantDevDesconectadosLAN'] = mintic_03['trafico.totales.cantDevLAN'] - mintic_03['trafico.totales.cantDevConectadosLAN']

    mintic_03.fillna({'trafico.totales.cantDevConectadosWAN':0,
                      'trafico.totales.cantDevDesconectadosWAN':0,
                      'trafico.totales.cantDevConectadosLAN':0,
                      'trafico.totales.cantDevDesconectadosLAN':0,
                      'trafico.totales.sitio.traficoIN':0,
                      'trafico.totales.sitio.traficoOUT':0,
                      'trafico.totales.sitio.consumoGB':0
                      },inplace=True)
    mintic_03['trafico.totales.cantDev'] = mintic_03['trafico.totales.cantDevLAN'] + mintic_03['trafico.totales.cantDevWAN']
    mintic_03[['trafico.totales.cantDev','trafico.totales.cantDevConectadosWAN','trafico.totales.cantDevDesconectadosWAN','trafico.totales.cantDevConectadosLAN','trafico.totales.cantDevDesconectadosLAN']] = mintic_03[['trafico.totales.cantDev','trafico.totales.cantDevConectadosWAN','trafico.totales.cantDevDesconectadosWAN','trafico.totales.cantDevConectadosLAN','trafico.totales.cantDevDesconectadosLAN']].astype(int)

    mintic_03['trafico.totales.cantDevConectados'] = mintic_03['trafico.totales.cantDevConectadosWAN'] + mintic_03['trafico.totales.cantDevConectadosLAN']
    mintic_03['trafico.totales.cantDevDesconectados'] = mintic_03['trafico.totales.cantDevDesconectadosLAN'] + mintic_03['trafico.totales.cantDevDesconectadosWAN']
    mintic_03[['trafico.totales.cantDevConectados'
              ,'trafico.totales.cantDevDesconectados'
              ,'trafico.totales.cantDevWAN'
              ,'trafico.totales.cantDevLAN']] = mintic_03[['trafico.totales.cantDevConectados'
                                                          ,'trafico.totales.cantDevDesconectados'
                                                          ,'trafico.totales.cantDevWAN'
                                                          ,'trafico.totales.cantDevLAN']].astype(int)    
    mintic_03['@timestamp'] = now.isoformat()
    
    
    salida = helpers.bulk(es, doc_generator_dis(mintic_03))
    
    print("Fecha: ", now,"- Datos Dispositivos Conectados en indice principal:",salida[0])
except:
    print("Fecha: ", now,"- No se insertaron datos de dispositivos conectados en indice principal")

In [ ]:
mintic_03.dtypes

### Guardando fecha para control de ejecución

* Se actualiza la fecha de control. Si el calculo supera la fecha hora actual, se asocia esta ultima.

In [ ]:
fecha_ejecucion = (datetime.strptime(fecha_max_mintic, '%Y-%m-%d %H:%M:%S')+timedelta(minutes=50)).strftime("%Y-%m-%d %H:%M:%S")[0:15] + '0:00'    

if fecha_ejecucion > str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00':
    fecha_ejecucion = str(now.strftime('%Y-%m-%d %H:%M:%S'))[0:15] + '0:00'
response = es.index(
        index = indice_control,
        id = 'jerarquia-tablero05',
        body = { 'jerarquia-tablero05': 'jerarquia-tablero05','tablero05.fechaControl' : fecha_ejecucion}
)
print("actualizada fecha control de ejecucion:",fecha_ejecucion)